## CL Dice & Betti Number

In [ ]:
from skimage.morphology import skeletonize, skeletonize_3d
import numpy as np

def cl_score(v, s):
    """[this function computes the skeleton volume overlap]
    Args:
        v ([bool]): [image]
        s ([bool]): [skeleton]
    Returns:
        [float]: [computed skeleton volume intersection]
    """
    return np.sum(v*s)/np.sum(s)


def clDice(v_p, v_l):
    """[this function computes the cldice metric]
    Args:
        v_p ([bool]): [predicted image]
        v_l ([bool]): [ground truth image]
    Returns:
        [float]: [cldice metric]
    """
    if len(v_p.shape)==2:
        tprec = cl_score(v_p,skeletonize(v_l))
        tsens = cl_score(v_l,skeletonize(v_p))
    elif len(v_p.shape)==3:
        tprec = cl_score(v_p,skeletonize_3d(v_l))
        tsens = cl_score(v_l,skeletonize_3d(v_p))
    return 2*tprec*tsens/(tprec+tsens)

In [ ]:
import numpy as np
import sys
import matplotlib.image as mpimg
from functools import reduce
import gudhi


def compute_persistence_diagram(matrix,min_pers=0,i=5):

    """
        Given a matrix representing a nii image compute the persistence diagram by using the Gudhi library (link)
        :param matrix: matrix encoding the nii image
        :type matrix: np.array
        :param min_pers: minimum persistence interval to be included in the persistence diagram
        :type min_pers: Integer
        :returns: Persistence diagram encoded as a list of tuples [d,x,y]=p where
            * d: indicates the dimension of the d-cycle p
            * x: indicates the birth of p
            * y: indicates the death of p
    """
    #save the dimenions of the matrix
    dims = matrix.shape
    size = reduce(lambda x, y: x * y, dims, 1)

    #create the cubica complex from the image
    cubical_complex = gudhi.CubicalComplex(dimensions=dims,top_dimensional_cells=np.reshape(matrix.T,size))
    #compute the persistence diagram
    if i == 5:
        pd = cubical_complex.persistence(homology_coeff_field=2, min_persistence=min_pers)
        return np.array(map(lambda row: [row[1][0],row[1][1]], pd))
    else:
        pd = cubical_complex.persistence(homology_coeff_field=2, min_persistence=min_pers)
        pd = cubical_complex.persistence_intervals_in_dimension(i)
        return np.array(list(map(lambda row: [row[0],row[1]], pd)))

In [ ]:
import sys
import numpy as np

def betti_number(imagely,imagefile):
	imagely = mpimg.imread(imagefile)
	imagely = imagely.detach().cpu().clone().numpy()
	width,height = imagely.shape
	imagely[width - 1, :] = 0
	imagely[:, height - 1] = 0
	imagely[0, :] = 0
	imagely[:, 0] = 0
	temp = compute_persistence_diagram(imagely, i = 1)
	betti_number = len(temp)
	# print (betti_number)
	return betti_number

In [ ]:
import sys
#sys.path.append('./source/')
import numpy as np
import matplotlib.image as mpimg
import os
#import ext_libs.Gudhi as gdh
#import pd
#import histo_image as hi

#imagely_copy = mpimg.imread('output_IMG_1.png')
imagely_copy = mpimg.imread('retina.jpg')
print(imagely_copy.shape)
imagely = imagely_copy.copy()
width,height,dim = imagely.shape
imagely[width - 1, :] = 0
imagely[:, height - 1] = 0
imagely[0, :] = 0
imagely[:, 0] = 0
temp = compute_persistence_diagram(imagely, i = 1)
betti_number = len(temp)
print (betti_number)

## Hausdorff Distance & Average symmetric surface distance

In [ ]:
import numpy
from scipy.ndimage import _ni_support
from scipy.ndimage import distance_transform_edt, binary_erosion, generate_binary_structure
from scipy.ndimage import label, find_objects
from scipy.stats import pearsonr

def surface_distances(result, reference, voxelspacing=None, connectivity=1):
    """
    The distances between the surface voxel of binary objects in result and their
    nearest partner surface voxel of a binary object in reference.
    """
    result = numpy.atleast_1d(result.astype(numpy.bool))
    reference = numpy.atleast_1d(reference.astype(numpy.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = numpy.asarray(voxelspacing, dtype=numpy.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()
            
    # binary structure
    footprint = generate_binary_structure(result.ndim, connectivity)
    
    # test for emptiness
    if 0 == numpy.count_nonzero(result): 
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == numpy.count_nonzero(reference): 
        raise RuntimeError('The second supplied array does not contain any binary object.')    
            
    # extract only 1-pixel border line of objects
    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)
    
    # compute average surface distance        
    # Note: scipys distance transform is calculated only inside the borders of the
    #       foreground objects, therefore the input has to be reversed
    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]
    
    return sds

In [ ]:
def hd(result, reference, voxelspacing=None, connectivity=1):
    """
    Hausdorff Distance.
    
    Computes the (symmetric) Hausdorff Distance (HD) between the binary objects in two
    images. It is defined as the maximum surface distance between the objects.
    
    Parameters
    ----------
    result : array_like
        Input data containing objects. Can be any type but will be converted
        into binary: background where 0, object everywhere else.
    reference : array_like
        Input data containing objects. Can be any type but will be converted
        into binary: background where 0, object everywhere else.
    voxelspacing : float or sequence of floats, optional
        The voxelspacing in a distance unit i.e. spacing of elements
        along each dimension. If a sequence, must be of length equal to
        the input rank; if a single number, this is used for all axes. If
        not specified, a grid spacing of unity is implied.
    connectivity : int
        The neighbourhood/connectivity considered when determining the surface
        of the binary objects. This value is passed to
        `scipy.ndimage.generate_binary_structure` and should usually be :math:`> 1`.
        Note that the connectivity influences the result in the case of the Hausdorff distance.
        
    Returns
    -------
    hd : float
        The symmetric Hausdorff Distance between the object(s) in ```result``` and the
        object(s) in ```reference```. The distance unit is the same as for the spacing of 
        elements along each dimension, which is usually given in mm.
        
    Notes
    -----
    This is a real metric. The binary images can therefore be supplied in any order.
    """
    hd1 = surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

In [ ]:
def assd(result, reference, voxelspacing=None, connectivity=1):
    """
    Average symmetric surface distance.
    
    Computes the average symmetric surface distance (ASD) between the binary objects in
    two images.
    
    Parameters
    ----------
    result : array_like
        Input data containing objects. Can be any type but will be converted
        into binary: background where 0, object everywhere else.
    reference : array_like
        Input data containing objects. Can be any type but will be converted
        into binary: background where 0, object everywhere else.
    voxelspacing : float or sequence of floats, optional
        The voxelspacing in a distance unit i.e. spacing of elements
        along each dimension. If a sequence, must be of length equal to
        the input rank; if a single number, this is used for all axes. If
        not specified, a grid spacing of unity is implied.
    connectivity : int
        The neighbourhood/connectivity considered when determining the surface
        of the binary objects. This value is passed to
        `scipy.ndimage.generate_binary_structure` and should usually be :math:`> 1`.
        The decision on the connectivity is important, as it can influence the results
        strongly. If in doubt, leave it as it is.         
        
    Returns
    -------
    assd : float
        The average symmetric surface distance between the object(s) in ``result`` and the
        object(s) in ``reference``. The distance unit is the same as for the spacing of 
        elements along each dimension, which is usually given in mm.
    """
    assd = numpy.mean( (surface_distances(result, reference, voxelspacing, connectivity), surface_distances(reference, result, voxelspacing, connectivity)) )
    
    return assd